In [2]:
import ssl
import certifi
import warnings
ssl._create_default_https_context = lambda: ssl.create_default_context(
    cafile=certifi.where()
)
warnings.filterwarnings("ignore")

In [3]:
from unstructured.partition.auto import partition
from unstructured.partition.md import partition_md
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.html import partition_html
from unstructured.partition.json import partition_json
from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements
from unstructured.partition.image import partition_image

from unstructured.documents.elements import (
    Title,
    NarrativeText,
    Table,
    ListItem,
    Header,
    Footer
)
import json
from pathlib import Path
from langchain_community.document_loaders import UnstructuredCSVLoader


objc[803]: Class AVFFrameReceiver is implemented in both /Users/nilasark/Demo/GENAI/env/lib/python3.13/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x125f583a8) and /Users/nilasark/Demo/GENAI/env/lib/python3.13/site-packages/av/.dylibs/libavdevice.62.1.100.dylib (0x1253443a8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
objc[803]: Class AVFAudioReceiver is implemented in both /Users/nilasark/Demo/GENAI/env/lib/python3.13/site-packages/cv2/.dylibs/libavdevice.61.3.100.dylib (0x125f583f8) and /Users/nilasark/Demo/GENAI/env/lib/python3.13/site-packages/av/.dylibs/libavdevice.62.1.100.dylib (0x1253443f8). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.


In [4]:
html_path=str(list(Path('./SRCs').glob('*.html'))[0])
elements=partition(html_path)
titles=[e for e in elements if isinstance(e,Title)]

first_title=titles[0]
metadata=first_title.metadata
print(metadata.to_dict())

{'category_depth': 0, 'last_modified': '2026-01-11T12:18:53', 'languages': ['eng'], 'file_directory': 'SRCs', 'filename': 'me.html', 'filetype': 'text/html'}


Filter specific elements

In [5]:
def analyse_elements(elements):
    """To Filter Specific Elements"""
    element_count={}
    for element in elements:
        ele_name=type(element).__name__
        element_count[ele_name]=element_count.get(ele_name,0)+1


    return element_count

analyse_elements(elements)

{'Title': 6, 'NarrativeText': 4, 'Text': 2}

Filter the Table Elements

In [6]:
table_elements=[ele for ele in elements if isinstance(ele,NarrativeText)]

if table_elements:
    print(table_elements[0])
    metadata=table_elements[0].metadata
    print(metadata.to_dict())

Usually used for the main title of the page.
{'last_modified': '2026-01-11T12:18:53', 'languages': ['eng'], 'file_directory': 'SRCs', 'filename': 'me.html', 'filetype': 'text/html', 'parent_id': '3dcc681dec33a49268ec1af97227d1b3'}


PDF Parsing through Unstructured

In [8]:
from unstructured.partition.auto import partition
pdf_path='https://arxiv.org/pdf/1706.03762'
file_path=list(Path('SRCs').glob('*.pdf'))[0]
elements=partition(
    filename=file_path,
    strategy="auto",          # auto/fast/hi_res/ocer_only
   infer_table_structure=True
)

print(analyse_elements(elements))

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


{'Text': 34, 'Header': 3, 'NarrativeText': 80, 'Title': 26, 'Footer': 8, 'Image': 7, 'FigureCaption': 5, 'Formula': 5, 'ListItem': 43, 'Table': 4}


Compare The Fast and Auto Strategy

3 2 0 2
g u A 2
] L C . s c [
7 v 2 6 7 3 0 . 6 0 7 1 : v i X r a
Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalis...
Attention Is All You Need
Ashish Vaswani∗ Google Brain avaswani@google.com
Noam Shazeer∗ Google Brain noam@google.com
Niki Parmar∗ Google Research nikip@google.com
Jakob Uszkoreit∗ Google Research usz@google.com
Llion Jones∗ Google Research llion@google.com
Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu
Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com
Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The b...
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started the effort to evaluate this idea. Ashish, ...
†Work performed while at Google Brain. ‡Work 

In [16]:
from unstructured.partition.auto import partition
from pathlib import Path
file_path='SRCs'

file=list(Path(file_path).glob('*.pdf'))[0]

def compare_strategies(strategy):
    elements=partition(
    filename=file,
    strategy=strategy
    )
    ele_count={}
    for ele in elements:
        ele_name=type(ele).__name__
        ele_count[ele_name]=ele_count.get(ele_name,0)+1
    # text=ele.text[:150] +'...' if len(ele.text)>150 else ele.text
    # print(text)
    print(ele_count)

print(compare_strategies("auto"))
print('*'*40)
print(compare_strategies("fast"))



{'Text': 70, 'NarrativeText': 139, 'Title': 158, 'Footer': 15, 'ListItem': 11}
None
****************************************
{'Text': 70, 'NarrativeText': 139, 'Title': 158, 'Footer': 15, 'ListItem': 11}
None


Markdown Partition

In [14]:
from unstructured.partition.md import partition_md
from pathlib import Path

file_path=Path('SRCs')
file=list(file_path.glob('*.pdf'))[1]

elements=partition_md(
    filename=file,
    table_elements=True
)
print(len(elements))
# ele_count={}
# for ele in elements:
#     ele_name=type(ele).__name__
#     ele_count[ele_name]=ele_count.get(ele_name,0)+1
#     text=ele.text[:80] +'...' if len(ele.text)>80 else ele.text
#     print(f"{ele_name:15} | {text}")

# print(ele_count)



107


PPTX partition

In [ ]:
from unstructured.partition.md import partition_md
from pathlib import Path

file_path=Path('SRCs')
file=list(file_path.glob('*.pptx'))[0]

elements=partition_md(
    filename=file,
    table_elements=True
)
ele_count={}
for ele in elements:
    ele_name=type(ele).__name__
    ele_count[ele_name]=ele_count.get(ele_name,0)+1
    text=ele.text[:80] +'...' if len(ele.text)>80 else ele.text
    print(f"{ele_name:15} | {text}")

print(ele_count)



Unstructures OCR models

In [3]:
from unstructured.partition.auto import partition
from unstructured.partition.image import partition_image
from pathlib import Path
file_path=Path('SRCs')
file=list(file_path.glob('*.jpg'))[0]


elements=partition_image(
    filename=file,
    strategy='hi_res',
    hi_res_model_name='yolox',
    ocr_languages='eng'
)

for ele in elements:
    print(ele.text)

The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.
Only one of languages and ocr_languages should be specified. languages is preferred. ocr_languages is marked for deprecation.


aws —
AWS DATABASE BLOG
AWS positioned highest in executi the 2023 Gartner Magic Quadrant ' Cloud Database Management Syst
Read the blog post >


Scanned Docs

In [8]:
from unstructured.partition.auto import partition
from unstructured.partition.pdf import partition_pdf
from pathlib import Path

file_path=Path('SRCs')
file=list(file_path.glob('*.pdf'))[0]

elements=partition_pdf(
    filename=file,
    strategy='ocr_only',
    ocr_languages='eng'
)

print(len(elements))

The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.
Only one of languages and ocr_languages should be specified. languages is preferred. ocr_languages is marked for deprecation.


3


Langchain Integration

In [7]:
from langchain_unstructured import UnstructuredLoader
from pathlib import Path
file_path=Path('SRCs')
file=list(file_path.glob('*.pdf'))[1]

loader=UnstructuredLoader(file_path=file,strategy='fast')
docs=loader.load()
print(len(docs[0].page_content))





7


Langchain Unstructured

In [8]:
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_unstructured import UnstructuredLoader
from langchain_chroma import Chroma
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

Document Loaders

In [12]:

def document_loader(file_paths):
    all_docs=[]

    for files in file_paths:
        if Path(files).exists():
            loader=UnstructuredLoader(
                file_path=files,
                mode="elements",
                strategy="auto",
                languages=["eng"]
            )
            docs=loader.load()
            all_docs.extend(docs)
    return all_docs


file_paths=[
    'SRCs/AIAN.pdf',
    'SRCs/me.html'
]
documents=document_loader(file_paths)

print(documents)

[Document(metadata={'source': 'SRCs/AIAN.pdf', 'coordinates': {'points': ((16.34, 213.92000000000007), (16.34, 253.92000000000007), (36.34, 253.92000000000007), (36.34, 213.92000000000007)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'SRCs', 'filename': 'AIAN.pdf', 'languages': ['eng'], 'last_modified': '2026-01-30T10:43:20', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': 'fd4d3bde7b92bfd2753fcf14821eddb7'}, page_content='3 2 0 2'), Document(metadata={'source': 'SRCs/AIAN.pdf', 'coordinates': {'points': ((16.34, 258.9200000000001), (16.34, 308.36), (36.34, 308.36), (36.34, 258.9200000000001)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'SRCs', 'filename': 'AIAN.pdf', 'languages': ['eng'], 'last_modified': '2026-01-30T10:43:20', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'NarrativeText', 'element_id': '63755f77c3940d8cdb3be27ae0

Text Splitters

In [13]:
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks=splitter.split_documents(documents)

Embeddings and VectorDb

In [15]:
from langchain_community.vectorstores.utils import filter_complex_metadata

embeddings=OpenAIEmbeddings(
    model='text-embedding-3-small'
)
filtered_splits=filter_complex_metadata(chunks)

vector_db=Chroma(
    collection_name="new_vector",
    embedding_function=embeddings,
    persist_directory='Chroma',
)
vector_db.add_documents(filtered_splits)

retriever=vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":5
    }
)





INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


LLM initialize

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm=ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0
)

prompts=ChatPromptTemplate.from_template(
"""
Answer thw Questions Based on The Context

Question:{Question}
Context:{Context}

The Answer is
"""
)

chain=({"Context":retriever,"Question":RunnablePassthrough()}|prompts|llm|StrOutputParser())

print(chain.invoke("What is Transformer ?"))





INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The Transformer is a model architecture that relies entirely on an attention mechanism to establish global dependencies between input and output, eschewing recurrence. It allows for significant parallelization and can achieve state-of-the-art translation quality after being trained for a relatively short time on powerful hardware. The Transformer is notable for being the first transduction model to use self-attention for computing representations of its input and output without relying on sequence-aligned RNNs or convolutional layers. Its architecture consists of stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. Additionally, it employs multi-head attention in various ways.


Error Handling

In [ ]:

def safe_partition(file_path, **kwargs):
    """
    Safely partition a document with error handling.
    
    Args:
        file_path: Path to the document
        **kwargs: Additional arguments for partition()
    
    Returns:
        Tuple of (elements, error_message)
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            return None, f"File not found: {file_path}"
        
        # Check file size (warn for large files)
        file_size = os.path.getsize(file_path)
        if file_size > 100 * 1024 * 1024:  # 100 MB
            print(f"Warning: Large file ({file_size / 1024 / 1024:.1f} MB)")
        
        # Partition the document
        elements = partition(filename=file_path, **kwargs)
        
        return elements, None
        
    except FileNotFoundError:
        return None, "File not found"
    except PermissionError:
        return None, "Permission denied"
    except Exception as e:
        return None, f"Unexpected error: {str(e)}"

# Test with various scenarios
test_files = [
    "sample_documents/sample.html",
    "nonexistent_file.pdf",
    "sample_documents/sample.md",
]

print("Safe Partition Results:")
print("="*60)

for file_path in test_files:
    elements, error = safe_partition(file_path)
    
    if error:
        print(f"\n{file_path}: ERROR - {error}")
    else:
        print(f"\n{file_path}: SUCCESS - {len(elements)} elements")